In [13]:
#Code to mount Google Drive at Colab Notebook instance
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Workaround to avoid following error at notebook
# NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [24]:
!pip install numpy==1.23.5


  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chainlit 1.1.306 requires numpy<2.0,>=1.26; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.


In [15]:
# Huggingface libraries to run LLM.
#!pip install -q -U numpy==1.23.5
!pip install -q -U transformers==4.40.2
!pip install -q -U accelerate==0.30.1
!pip install -q -U bitsandbytes==0.43.1
!pip install -q -U huggingface_hub==0.23.0

#LangChain related libraries
!pip install -q -U langchain==0.1.2

#Open-source pure-python PDF library capable of splitting, merging, cropping,
#and transforming the pages of PDF files
!pip install -q -U pypdf==4.2.0

#Python framework for state-of-the-art sentence, text and image embeddings.
!pip install -q -U sentence-transformers==2.7.0

# FAISS Vector Databses specific Libraries
!pip install -q -U faiss-gpu==1.7.2

In [16]:
!pip install -q -U chainlit

In [2]:
import numpy as np
print(np.__version__)


1.23.5


In [17]:
import chainlit as cl
from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader, CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from huggingface_hub import notebook_login
# Setup your environment
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))


Device: cuda
Tesla T4


In [18]:
# Log in to Huggingface
notebook_login()


In [19]:
# Load your model and tokenizer
origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True,
                                             quantization_config=bnb_config,
                                             device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [20]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=300,
    temperature=0.3,
    do_sample=True,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [21]:
'''
#Load and process your documents
loader = CSVLoader(file_path='/content/drive/MyDrive/Opportunities.csv')
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunked_docs = text_splitter.split_documents(data)
'''
# Create embeddings and vector store
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
# Define paths to your CSV files
csv_files = [
    '/content/drive/MyDrive/Opportunities.csv',
    '/content/drive/MyDrive/Courses.csv',
    '/content/drive/MyDrive/Events.csv'
]

# Load and combine data from multiple CSV files
combined_data = []
for file in csv_files:
    loader = CSVLoader(file_path=file)
    data = loader.load()
    combined_data.extend(data)  # Combine data into a single list

# Split the documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunked_docs = text_splitter.split_documents(combined_data)

# Create embeddings and vector store
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
db = FAISS.from_documents(chunked_docs, embeddings)

# Create retriever and QA chain
retriever = db.as_retriever(search_type="similarity", search_kwargs={'k': 4})

In [23]:
qa_chain = ConversationalRetrievalChain.from_llm(mistral_llm, retriever, return_source_documents=False)


In [33]:
import sys
chat_history = []
query = input('Prompt: ')
result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
print('Answer: ' + result['answer'] + '\n')
chat_history.append((query, result['answer']))

Prompt: tell me the latest opportunity you see
Answer:  The latest opportunity I see is the Business Development Executive- Global Saas position posted on Shortlist. It has a deadline of 2024-01-27 21:00:00.



In [12]:
!lsof -i :5001

COMMAND     PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
pt_main_t 30831 root   80u  IPv4 999326      0t0  TCP *:5001 (LISTEN)


In [25]:
from flask import Flask, request, jsonify
#from flask_cors import CORS
import threading

app = Flask(__name__)
#CORS(app)

@app.route('/query', methods=['POST'])
def query():
    data = request.json
    question = data['question']
    chat_history = data.get('chat_history', [])

    # Ensure chat_history is in the correct format
    formatted_chat_history = [(entry[0], entry[1]) for entry in chat_history]

    try:
        result = qa_chain.invoke({'question': question, 'chat_history': formatted_chat_history})
        return jsonify({'answer': result['answer']})
    except Exception as e:
        # Handle exceptions and return a meaningful message
        return jsonify({'error': str(e)}), 500

def run_flask():
    app.run(host='0.0.0.0', port=5003)

# Run Flask in a separate thread
threading.Thread(target=run_flask, daemon=True).start()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5003
 * Running on http://172.28.0.12:5003
INFO:werkzeug:Press CTRL+C to quit


In [26]:
!ssh -R 80:localhost:5003 serveo.net


Forwarding HTTP traffic from https://ffa87398fe8de1d7654fd29f4a1efadf.serveo.net
HTTP request from 41.175.88.142 to https://ffa87398fe8de1d7654fd29f4a1efadf.serveo.net/query


INFO:werkzeug:127.0.0.1 - - [24/Jul/2024 17:49:12] "POST /query HTTP/1.1" 200 -


HTTP request from 41.175.88.142 to https://ffa87398fe8de1d7654fd29f4a1efadf.serveo.net/query


INFO:werkzeug:127.0.0.1 - - [24/Jul/2024 17:50:33] "POST /query HTTP/1.1" 200 -


HTTP request from 41.175.88.142 to https://ffa87398fe8de1d7654fd29f4a1efadf.serveo.net/query


INFO:werkzeug:127.0.0.1 - - [24/Jul/2024 18:20:11] "POST /query HTTP/1.1" 200 -


Port forwarding for ffa87398fe8de1d7654fd29f4a1efadf.serveo.net expired


In [40]:
!lsof -i :8000